# Kings County Housing Prices Bakeoff

Below are a list of steps that you should take while trying to complete your bake-off entry.

## Step 1: Read in Data

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import norm 
import math
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
import fiona
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
import seaborn as sns
plt.style.use('seaborn')
sns.set(style="white")

In [3]:
hf = pd.read_csv('kc_house_data_train.csv')
zipfile = "Zip_Codes-shp"
street_map = gpd.read_file(zipfile)
crs = {'init': 'epsg:4326'}

In [4]:
hf1 = hf.copy()

In [5]:
zip_grade = pd.read_csv('Niche.csv')
hf = zip_grade.set_index('zipcode').join(hf.set_index('zipcode'))

In [6]:
hf = hf.reset_index()

In [7]:
hf = hf.dropna(subset = ['id'])

## Step 2: Exploratory Data Analysis 
    
Become familiar with the data.  Look to see if there are any extreme values.  

Additionally create data visualizations to determine if there are any relationships between your features and your target variables.  

In [ ]:
# Plot Histogram
sns.distplot(hf['price'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(hf['price'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

fig = plt.figure()
res = stats.probplot(hf['price'], plot=plt)
plt.show()

print("Skewness: %f" % hf['price'].skew())
print("Kurtosis: %f" % hf['price'].kurt())

In [ ]:
sns.regplot(hf['price'],hf['bathrooms'], scatter_kws={"color": "black"}, line_kws={"color": "red"})

In [ ]:
fig, axes = plt.subplots(7,2, figsize = (20, 20))


sns.boxplot(ax = axes[0,0], data = hf['bedrooms'], orient = 'h')
sns.boxplot(ax = axes[0,1], data = hf['bathrooms'], orient = 'h')
sns.boxplot(ax = axes[1,0], data = hf['sqft_living'], orient = 'h')
sns.boxplot(ax = axes[1,1], data = hf['sqft_lot'], orient = 'h')
sns.boxplot(ax = axes[2,0], data = hf['floors'], orient = 'h')
sns.boxplot(ax = axes[2,1], data = hf['grade'], orient = 'h')
sns.boxplot(ax = axes[3,0], data = hf['sqft_above'], orient = 'h')
sns.boxplot(ax = axes[3,1], data = hf['sqft_basement'], orient = 'h')
sns.boxplot(ax = axes[4,0], data = hf['yr_built'], orient = 'h')
sns.boxplot(ax = axes[4,1], data = hf['yr_renovated'], orient = 'h')
sns.boxplot(ax = axes[5,0], data = hf['zipcode'], orient = 'h')
sns.boxplot(ax = axes[5,1], data = hf['sqft_living15'], orient = 'h')
sns.boxplot(ax = axes[6,0], data = hf['sqft_lot15'], orient = 'h')


plt.show

In [ ]:
numerical = ['price', 'sqft_living', 'sqft_lot', 'view',
             'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']

categorical = ['bedrooms', 'bathrooms', 'floors', 'waterfront', 'condition', 'grade', 'zipcode']

hf1 = hf1[numerical + categorical]

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(30, 20))
for var, subplot in zip(categorical, ax.flatten()):
    sns.boxplot(x=var, y='price', data=hf1, ax=subplot)

In [ ]:
def correlation_heatmap(hf):
    _,ax=plt.subplots(figsize=(25,20))
    colormap=sns.diverging_palette(220,10,as_cmap=True)
    sns.heatmap(hf.corr(),annot=True,cmap=colormap)
    
correlation_heatmap(hf)

## Step 3: Clean up any issues (extreme values, etc.) with the data.  

Remember that you can't just delete rows with extreme values. Similar observations might be present in the holdout data set, and you can't just delete those rows and not have a prediction for it. 

In [8]:
sq_lot_mean = hf['sqft_lot'].mean()
print(sq_lot_mean)
sq_lot_std3 = hf['sqft_lot'].std()*3
sq_lot_mstd = sq_lot_mean + sq_lot_std3

sq_lot_mstd 

15243.3998843262


142157.2712491516

In [9]:
sq_living_mean = hf['sqft_living'].mean()
print(sq_living_mean)
sq_living_std3 = hf['sqft_living'].std()*3
sq_living_mstd = sq_living_mean + sq_living_std3
sq_living_mstd

2081.4646038172355


4841.520222012694

In [10]:
sq_above_mean = hf['sqft_above'].mean()
print(sq_above_mean)
sq_above_std3 = hf['sqft_above'].std()*3
sq_above_mstd = sq_above_mean + sq_above_std3

sq_above_mstd

1789.306015037594


4277.101336107848

In [11]:
sq_base_mean = hf['sqft_basement'].mean()
print(sq_base_mean)
sq_base_std3 = hf['sqft_basement'].std()*3
sq_base_mstd = sq_base_mean + sq_base_std3
sq_base_mstd

292.15858877964143


1621.61421219585

In [12]:
def cap_sqft(row):
    if row['sqft_lot'] > sq_lot_mstd:
        row['sqft_lot'] = sq_lot_mstd
    if row['sqft_living'] > sq_living_mstd:
        row['sqft_living'] = sq_living_mstd
    if row['sqft_above'] > sq_above_mstd :
        row['sqft_above'] = sq_above_mstd
    if row['sqft_basement'] > sq_base_mstd :
        row['sqft_basement'] = sq_base_mstd 
    return row

In [13]:
hf = hf.apply(cap_sqft, axis = 1)

In [14]:
def zero_val_bed_bath(row):
    """
    Checking extreame number of rooms in the house
    """
    if row['bedrooms'] == 0:
        row['bedrooms'] = row['floors']
    if row['bathrooms'] < 1:
        row['bathrooms'] = 1
    if row['bedrooms'] > 10 :
        row['bedrooms'] = 10
    return row

In [15]:
hf = hf.apply(zero_val_bed_bath, axis = 1)

In [16]:
hf.drop(columns = ['zip_rank', 'Unnamed: 0', 'id', 'view', 'sqft_living15', 'sqft_lot15'], inplace = True)

In [17]:
def define_niche_grade(row):
    
    if row['niche_grade'] == 'A+':
        row['niche_grade'] = 1
    if row['niche_grade'] == 'A+ ':
        row['niche_grade'] = 1
    if row['niche_grade'] == 'A':
        row['niche_grade'] = 2
    if row['niche_grade'] == 'A-':
        row['niche_grade'] = 3
    if row['niche_grade'] == 'B+':
        row['niche_grade'] = 4
    if row['niche_grade'] == 'B':
        row['niche_grade'] = 5
    if row['niche_grade'] == 'B-':
        row['niche_grade'] = 6
    
    return row 

In [18]:
hf = hf.apply(define_niche_grade, axis = 1)

In [19]:
def define_school_grade(row):
    
    if row['school_grade'] == 'A+':
        row['school_grade'] = 1
    if row['school_grade'] == 'A+ ':
        row['school_grade'] = 1
    if row['school_grade'] == 'A ':
        row['school_grade'] = 2
    if row['school_grade'] == 'A':
        row['school_grade'] = 2
    if row['school_grade'] == 'A-':
        row['school_grade'] = 3
    if row['school_grade'] == 'A- ':
        row['school_grade'] = 3
    if row['school_grade'] == 'B+':
        row['school_grade'] = 4
    if row['school_grade'] == 'B':
        row['school_grade'] = 5
    if row['school_grade'] == 'B-':
        row['school_grade'] = 6
    if row['school_grade'] == 'C+':
        row['school_grade'] = 7
    
    return row 

In [20]:
hf = hf.apply(define_school_grade, axis = 1)

## Step 4: Generate new features that you think could be important.

After doing this, you will want to go back to steps 2 and 3 to investigate these new features.

In [21]:
hf['yr_updated'] = np.nan

In [22]:
def yr_update(row):
    
    if row['yr_renovated'] == 0:
        row['yr_updated'] = 2021 - row['yr_built']
    if row['yr_renovated'] != 0:
        row['yr_updated'] = 2021 - row['yr_built']
        
    return row 
    

In [23]:
hf = hf.apply(yr_update, axis = 1)

In [24]:
hf['percent_bedbath'] = np.nan
hf['has_golden_ratio'] = np.nan

In [25]:
def cal_ratio_range(row):
    golden_ratio = (2/3)
    golden_ratio_plus = golden_ratio + (golden_ratio * .10)
    golden_ratio_minus = golden_ratio - (golden_ratio * .10)
    
    if row['percent_bedbath'] <= golden_ratio_plus and row['percent_bedbath'] >= golden_ratio_minus:
            row['has_golden_ratio'] = 1
    else:
        row['has_golden_ratio'] = 0
    
    return row

In [26]:
hf = hf.apply(cal_ratio_range, axis = 1)

In [27]:
# Based off of bathrooms pros and housetipster 

def ratio_bed_bath(row):
    
    ratio_bed_bath = row['bathrooms'] / row['bedrooms']
    golden_ratio = (2/3)
    row['percent_bedbath'] = abs(golden_ratio - ratio_bed_bath) 
    
    return row

In [28]:
hf = hf.apply(ratio_bed_bath, axis = 1)

In [29]:
hf['ratio_liv_lot'] = np.nan

In [30]:
def ratio_living_lot(row):
    
    row['ratio_liv_lot'] = row['sqft_lot'] / row['sqft_living']
    return row
    

In [31]:
hf = hf.apply(ratio_living_lot, axis = 1)

## Step 5: Train-Test Split

If you plan on doing any scaling of your data, make sure it is done at the appropriate time. 

In [32]:
hf = pd.concat([hf, pd.get_dummies(hf['zipcode'])], 1)

In [33]:
hf = hf.drop(columns = 'zipcode')

In [34]:
hf = pd.concat([hf, pd.get_dummies(hf['grade'])], 1)

In [35]:
hf = hf.drop(columns = 'grade')

In [36]:
hf.columns = hf.columns.astype(str)

In [37]:
# lowest g: 1
# low g: 3 Falls short of minimum building standards. Normally cabin or inferior structure.

# dnmc: 4 Generally older, low quality construction. Does not meet code.

# poor: 5 Low construction costs and workmanship. Small, simple design.

# bare_min: 6 Lowest grade currently meeting building code. Low quality materials and simple designs.

# average: 7 Average grade of construction and design. Commonly seen in plats and older sub-divisions.

# above_avg: 8 Just above average in construction and design. Usually better materials in both the exterior and interior finish work.

# good: 9 Better architectural design with extra interior and exterior design and quality.

# high_qua: 10 Homes of this quality generally have high quality features. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.

# higher_qua: 11 Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.

# excellent qua: 12 Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.

# mansion: 13 Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.

In [38]:
hf = hf.rename(columns={'1': 'lowest_g', '3': 'low_g', '4':'dnmc', '5':'Poor', '6':'bare_min', 
                   '7':'average', '8':'above_avg', '9':'good', '10':'high_qua', '11':'higher_qua',
                   '12':'excellent_qua', '13':'mansion' })

In [39]:
# your code here
# waterfront times sqft_lot
hf = pd.concat([hf, pd.get_dummies(hf['waterfront'])], 1)

In [40]:
hf = hf.rename(columns={0: "No_Waterfront", 1: "Waterfront"})

In [41]:
hf['water_sqft_lot'] = np.nan

In [42]:
def water_lot(row):
    if row['waterfront'] == 1:
        row['water_sqft_lot'] = row['Waterfront'] * row['sqft_lot'] 
    if row['waterfront'] == 0:
        row['water_sqft_lot'] = 0
    return row

In [43]:
hf = hf.apply(water_lot, axis=1)

In [44]:
hf = hf.drop(columns='waterfront')

In [45]:
features = ['school_grade', 'population', 
             'bedrooms', 'bathrooms', 
            'sqft_living', 'sqft_lot', 'floors', 'condition', 'sqft_above', 
            'sqft_basement', 'yr_built', 'yr_renovated', 
            'percent_bedbath','has_golden_ratio', 'lowest_g','low_g', 'dnmc', 'Poor', 
            'bare_min', 'average', 'above_avg', 'good', 'high_qua', 'higher_qua', 
            'excellent_qua', 'mansion', 'No_Waterfront',  'Waterfront', 'water_sqft_lot',
            '98001', '98002', '98003', '98004', '98005', '98006',
           '98007', '98008', '98010', '98011', '98014', '98019', '98022', '98023',
           '98024', '98027', '98028', '98029', '98030', '98031', '98032', '98033',
           '98034', '98038', '98039', '98040', '98042', '98045', '98052', '98053',
           '98055', '98056', '98058', '98059', '98065', '98070', '98072', '98074',
           '98075', '98077', '98092', '98102', '98103', '98105', '98106', '98107',
           '98108', '98109', '98112', '98115', '98116', '98117',
           '98118', '98119', '98122', '98125', '98126', '98133', '98136', '98144',
           '98146', '98148', '98155', '98166', '98168', '98177', '98178', '98188',
           '98198', '98199']

hf_features = hf[features]
target = hf['price']

In [46]:
def model_test(df, features, target):
    df_features = df[features]
    X_train, X_test, y_train, y_test = train_test_split(df_features, target, random_state=34,test_size=0.2)
    #instantiate a linear regression object
    lm = linear_model.LinearRegression()

    #fit the linear regression to the data
    lm = lm.fit(X_train, y_train)
    
    y_train_pred = lm.predict(X_train)
    
    train_mae = metrics.mean_absolute_error(y_train, y_train_pred)
    train_mse = metrics.mean_squared_error(y_train, y_train_pred)
    train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
   
    # Test Set
    y_pred = lm.predict(X_test)
    
    #test_mae = metrics.mean_absolute_error(y_test, y_pred)
    test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

    #print('Mean Absolute Error:' + str(metrics.mean_absolute_error(y_test, y_pred)))
    #print('Mean Squared Error:' + str(metrics.mean_squared_error(y_test, y_pred)))
    print('Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))

In [47]:
model_test(hf, features, target)

Training:  160137 vs. Testing:  155432


In [48]:
features = ['niche_grade', 'yr_updated', 'ratio_liv_lot']

hf_poly = hf[features]
target = hf['price']

In [49]:
poly = PolynomialFeatures(degree=2, include_bias=False)

In [50]:
poly_data = poly.fit_transform(hf_poly)

In [51]:
poly_columns = poly.get_feature_names(hf_poly.columns)

In [52]:
df_poly = pd.DataFrame(poly_data, columns=poly_columns)

In [53]:
list(df_poly)

['niche_grade',
 'yr_updated',
 'ratio_liv_lot',
 'niche_grade^2',
 'niche_grade yr_updated',
 'niche_grade ratio_liv_lot',
 'yr_updated^2',
 'yr_updated ratio_liv_lot',
 'ratio_liv_lot^2']

In [54]:
final_df = pd.concat([hf_features, df_poly], 1)

In [55]:
final_df.shape

(17290, 108)

## Step 6: Utilize some different feature selection techniques before or in conjuction with fitting your models.

In [56]:
effect_feat = ['school_grade', 'population', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'yr_updated', 'has_golden_ratio', 'Poor', 'bare_min',
       'average', 'good', 'high_qua', 'higher_qua', 'excellent_qua', 'mansion',
       'No_Waterfront', 'Waterfront', 'water_sqft_lot', '98001', '98002',
       '98003', '98004', '98005', '98006', '98022', '98023', '98030', '98031',
       '98032', '98033', '98038', '98039', '98040', '98042', '98053', '98055',
       '98058', '98074', '98075', '98077', '98092', '98102', '98105', '98106',
       '98108', '98109', '98112', '98118', '98119', '98133', '98146', '98155',
       '98168', '98178', '98188', '98198', '98199', 'niche_grade',
       'yr_updated', 'ratio_liv_lot', 'niche_grade^2',
       'niche_grade yr_updated', 'niche_grade ratio_liv_lot',
       'yr_updated ratio_liv_lot']

In [57]:
len(effect_feat)

70

In [58]:
final_df = final_df[effect_feat]

In [59]:
final_df.shape

(17290, 70)

In [60]:
list(final_df)

['school_grade',
 'population',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'yr_updated',
 'has_golden_ratio',
 'Poor',
 'bare_min',
 'average',
 'good',
 'high_qua',
 'higher_qua',
 'excellent_qua',
 'mansion',
 'No_Waterfront',
 'Waterfront',
 'water_sqft_lot',
 '98001',
 '98002',
 '98003',
 '98004',
 '98005',
 '98006',
 '98022',
 '98023',
 '98030',
 '98031',
 '98032',
 '98033',
 '98038',
 '98039',
 '98040',
 '98042',
 '98053',
 '98055',
 '98058',
 '98074',
 '98075',
 '98077',
 '98092',
 '98102',
 '98105',
 '98106',
 '98108',
 '98109',
 '98112',
 '98118',
 '98119',
 '98133',
 '98146',
 '98155',
 '98168',
 '98178',
 '98188',
 '98198',
 '98199',
 'niche_grade',
 'yr_updated',
 'ratio_liv_lot',
 'niche_grade^2',
 'niche_grade yr_updated',
 'niche_grade ratio_liv_lot',
 'yr_updated ratio_liv_lot']

## Step 7: Evaluate your different models in order to determine the best model overall.

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

alpha = 0.05 
#ANOVA Test Setup
formula = 'price~C(floors)'
lm_condition = smf.ols(formula, hf).fit()
anova_condition = sm.stats.anova_lm(lm_condition, type=2)
if anova_condition["PR(>F)"][0] < alpha:
    print("The number of floors has a statistically significant impact on average property value")
    print("Conditions F-statisic Probability: ", anova_condition["PR(>F)"][0])

In [ ]:
is_waterfront = hf[(hf['Waterfront'] == 1)]
waterfront_price = is_waterfront.price
no_waterfront = hf[(hf['Waterfront'] == 0)]
notwaterfront_price = no_waterfront.price
alpha = 0.05
waterfront_p_val = stats.ttest_ind(waterfront_price, notwaterfront_price, equal_var=False)[1]
print("Waterfront vs No Waterfront T-test P Value: ", waterfront_p_val)
if waterfront_p_val < alpha:
    print("The P value is less than alpha, reject null-hypothesis")

In [ ]:
alpha = 0.05 
#ANOVA Test Setup
formula = 'price~C(bedrooms)'
lm_condition = smf.ols(formula, hf).fit()
anova_condition = sm.stats.anova_lm(lm_condition, type=2)
if anova_condition["PR(>F)"][0] < alpha:
    print("The number of bedrooms has a statistically significant impact on average property value")
    print("Conditions F-statisic Probability: ", anova_condition["PR(>F)"][0])

## Step 8:  Refit your best model to the entire dataset.

In [ ]:
final_df.describe()

## Step 9: Save your final model using pickle.

https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

In [61]:
import pickle

In [62]:
def scale_fit_pickle_origin(df_features, target):
    """
    Scaling df with features,
    Fit linear model with scaled features
    Create pickle file with Scaler and Model
    params: 
            df_features - most important features 
            target - Series
    """
#     scaler = StandardScaler()
#     # fit the scaler to the training data
#     scaler.fit(df_features)
#     #transform the training data
#     scaled_data = scaler.transform(df_features)
#     #create dataframe
#     df_features_scaled = pd.DataFrame(data=scaled_data, columns=df_features.columns, index=df_features.index)
    lm_final = LinearRegression()
#     #fit the linear regression to the data
    lm_final = lm_final.fit(df_features, target)
    pickle_out = open("model.pickle","wb")
    pickle.dump(lm_final, pickle_out)
    pickle_out.close()
#     pickle_out = open('scaler.pickle', "wb")
#     pickle.dump(scaler, pickle_out)
#     pickle_out.close()
    return print(' CONGATS !!! You sucessfuly created you pickles for SCALER and MODEL')

In [63]:
scale_fit_pickle_origin(final_df, hf['price'])

 CONGATS !!! You sucessfuly created you pickles for SCALER and MODEL
